In [80]:
import tensorflow as tf
from transformers import *
from transformers import BertModel, BertConfig
from nltk.tokenize import sent_tokenize 
import torch
import matplotlib.pyplot as plt
from google.cloud import speech_v1p1beta1 as speech
from google.cloud import speech_v1 as speech
from google.cloud.speech_v1 import enums
from google.cloud.speech_v1 import types
import os
import re
import pandas as pd
import time
import numpy as np

In [81]:

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

def analise_audio_sentimento_google(texto):
    client = language.LanguageServiceClient().from_service_account_json("xxxxxxxxx.json")
    document = types.Document(
        content=texto,
        type=enums.Document.Type.PLAIN_TEXT)
    encoding_type = enums.EncodingType.UTF8
    annotations = client.analyze_sentiment(document=document,encoding_type=encoding_type,timeout=20)
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    return score,magnitude



In [195]:
atendente='hola, chico, como estás ?'
cliente='no estoy muy bien mi novia esta enferma'

In [196]:
analise_audio_sentimento_google(atendente)

(0.10000000149011612, 0.10000000149011612)

In [201]:
analise_audio_sentimento_google(cliente)[0]

-0.5

In [202]:
import nltk
def extract(sentence):
    
    sent=nltk.sent_tokenize(sentence)
    for sentence in sent:
        tokenized_text = nltk.word_tokenize(sentence)
    return re.findall(r'\w+', str(tokenized_text))

In [203]:
extract(atendente)

['hola', 'chico', 'como', 'estás']

In [204]:
extract(cliente)

['no', 'estoy', 'muy', 'bien', 'mi', 'novia', 'esta', 'enferma']

In [114]:
def embed(phrase):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
    model = BertModel.from_pretrained('bert-base-multilingual-uncased')
    input_ids = torch.tensor(tokenizer.encode(phrase)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    return last_hidden_states[-1]

In [119]:
np.sum(embed(extract(atendente)[0]).detach().cpu().numpy(),axis=0)

array([-7.26488471e-01,  1.28807068e+00,  2.27688894e-01,  1.11506486e+00,
       -2.12813568e+00,  1.05410290e+00, -2.12662399e-01,  4.23673928e-01,
       -5.00859165e+00,  2.32377082e-01, -1.05076432e+00, -2.35145807e+00,
        1.25793368e-01,  4.55494642e-01,  2.31674433e-01,  2.56057948e-01,
       -1.26149249e+00,  4.61393893e-01, -2.19370931e-01, -4.14451420e-01,
       -5.19407809e-01,  4.79118973e-01,  4.27455157e-01, -6.97281837e-01,
        2.08984065e+00,  5.69295943e-01,  7.36486435e-01, -1.24293581e-01,
       -4.26134014e+00, -8.46360981e-01, -6.30323648e-01,  6.99398935e-01,
       -9.01297569e-01, -1.55593574e-01, -1.09106168e-01, -7.16887295e-01,
       -4.55175519e-01,  3.66359973e+00, -5.65844327e-02, -4.45849299e-01,
       -8.65801424e-02, -4.41997170e-01, -7.43282437e-01,  4.36751783e-01,
       -1.13893497e+00, -7.11549580e-01,  3.99369299e-01,  1.05178821e+00,
       -2.72150725e-01, -4.61775780e-01, -1.49343479e+00,  1.59592938e+00,
        7.70732045e-01, -

In [125]:
import re
def embed_tf(phrase):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
    model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')
    input_ids = tf.constant(tokenizer.encode(phrase))[None, :]  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    return last_hidden_states

In [171]:
def vetor_palavra(parte):
    return np.sum(embed_tf(extract(parte)),axis=1)[0].reshape(-1,1)


In [172]:
from bertviz import head_view
from transformers import BertTokenizer, BertModel

In [173]:
def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        attention = model(input_ids, token_type_ids=token_type_ids)[-1]
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        attention = model(input_ids)[-1]
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    return attention, tokens, sentence_b_start

In [174]:
model_version = 'bert-base-multilingual-uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

output=show_head_view(model, tokenizer, atendente, cliente)

In [175]:
output[0][11].detach().cpu().numpy().reshape(12,441)[11]

array([2.17869738e-03, 1.31277796e-02, 2.80366722e-03, 6.30797772e-03,
       7.25429226e-03, 8.38949997e-03, 1.19936641e-03, 2.64771841e-03,
       3.44391563e-03, 2.59332452e-03, 1.24295813e-03, 1.02117436e-03,
       1.06551277e-03, 5.95627865e-03, 1.32061902e-03, 2.68675946e-03,
       1.84184359e-03, 4.40428732e-03, 7.39372382e-03, 2.16843793e-03,
       9.20952201e-01, 4.03298181e-04, 4.14395452e-01, 1.97663829e-02,
       4.51690005e-03, 3.17852870e-02, 4.03914414e-03, 1.99712981e-02,
       4.00633737e-03, 6.29308773e-03, 1.54288486e-03, 1.04138218e-02,
       2.09533959e-03, 5.97261183e-04, 8.21984746e-03, 1.37330440e-03,
       1.93390029e-03, 4.74563986e-03, 7.46482518e-04, 5.61155472e-03,
       5.88819059e-03, 4.51654702e-01, 3.27000598e-04, 1.98237645e-03,
       5.26760379e-03, 2.04942678e-03, 3.15776933e-03, 1.38597435e-03,
       5.75360260e-04, 1.46670907e-03, 6.97031268e-04, 1.45648979e-03,
       2.70402030e-04, 1.42113640e-04, 6.78966171e-05, 5.82479523e-04,
      

In [176]:
# modelo cada #palavra relaciona com as outras 11 #palavras


def relacionamento(A,B):
    return output[0][A].detach().cpu().numpy().reshape(12,441)[B].reshape(1,441)[0]

In [210]:
import warnings
warnings.filterwarnings('ignore')

In [253]:
def peso(palavra_cliente):
    relas=[]
    for i in range(0,len(extract(atendente))):
        relas.append(np.sum(np.sum(vetor_palavra(atendente)[i])*relacionamento(i,palavra_cliente)))
    return extract(atendente)[np.argmax(relas)],extract(cliente)[palavra_cliente-len(extract(atendente))]

In [254]:
extract(cliente)

['no', 'estoy', 'muy', 'bien', 'mi', 'novia', 'esta', 'enferma']

In [ ]:
result=[]
for k in range(len(extract(atendente)),len(extract(atendente))+len(extract(cliente))):
    result.append(peso(k))

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
result